## 1. 导入模块

In [ ]:
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# 添加 src 到路径
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from generator.core import DataGenerator
from generator.distributions import DistributionEngine
from llm.parser import ConfigParser
from utils.file_io import FileIO

## 2. 创建配置

In [ ]:
# 定义数据集配置
config = {
    "dataset_meta": {
        "num_transactions": 1000,
        "num_items": 100,
        "density": 0.1,
        "avg_transaction_len": 10
    },
    "distribution_config": {
        "method": "zipf",
        "params": {
            "alpha": 1.2
        }
    },
    "pattern_injection": [
        {
            "id": "frequent_combo",
            "items": [5, 10, 15],
            "target_support": 0.1,
            "noise_ratio": 0.05
        },
        {
            "id": "rare_pattern",
            "items": [20, 25],
            "target_support": 0.05,
            "noise_ratio": 0.1
        }
    ]
}

print(json.dumps(config, indent=2))

## 3. 验证配置

In [ ]:
# 使用 ConfigParser 验证
parser = ConfigParser(strict_mode=False)
validated_config = parser.parse(config)

print("配置验证通过！")
print(f"事务数: {validated_config['dataset_meta']['num_transactions']}")
print(f"物品数: {validated_config['dataset_meta']['num_items']}")
print(f"分布类型: {validated_config['distribution_config']['method']}")

## 4. 生成数据

In [ ]:
# 创建生成器
generator = DataGenerator(validated_config)

# 生成数据
data = generator.generate(seed=42)

print(f"生成的数据形状: {data.shape}")
print(f"数据类型: {data.dtype}")
print(f"总条目数: {data.sum()}")

## 5. 查看统计信息

In [ ]:
# 获取统计信息
stats = generator.get_statistics()

print("\n=" * 60)
print("数据集统计")
print("=" * 60)
print(f"事务数: {stats['num_transactions']}")
print(f"物品数: {stats['num_items']}")
print(f"实际密度: {stats['actual_density']:.2%}")
print(f"平均事务长度: {stats['avg_transaction_length']:.2f}")
print(f"标准差: {stats['std_transaction_length']:.2f}")
print(f"最小长度: {stats['min_transaction_length']}")
print(f"最大长度: {stats['max_transaction_length']}")

print("\n注入的模式:")
for pattern in stats['injected_patterns']:
    print(f"  {pattern['id']}:")
    print(f"    物品: {pattern['items']}")
    print(f"    目标支持度: {pattern['target_support']:.2%}")
    print(f"    实际支持度: {pattern['actual_support']:.2%}")

## 6. 可视化分布

In [ ]:
# 可视化事务长度分布
transaction_lengths = data.sum(axis=1)

plt.figure(figsize=(12, 5))

# 直方图
plt.subplot(1, 2, 1)
plt.hist(transaction_lengths, bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('事务长度')
plt.ylabel('频率')
plt.title('事务长度分布')
plt.axvline(transaction_lengths.mean(), color='red', linestyle='--', label=f'平均值: {transaction_lengths.mean():.2f}')
plt.legend()

# 物品频率
item_frequencies = data.sum(axis=0)
plt.subplot(1, 2, 2)
plt.bar(range(len(item_frequencies)), sorted(item_frequencies, reverse=True))
plt.xlabel('物品 ID（按频率排序）')
plt.ylabel('出现次数')
plt.title('物品频率分布（Zipf）')

plt.tight_layout()
plt.show()

## 7. 保存数据

In [ ]:
# 保存为 SPMF 格式
output_path = "../data/processed/example_dataset.spmf"
generator.to_spmf(output_path)

print(f"数据已保存到: {output_path}")

# 读取并验证
transactions = FileIO.read_spmf(output_path)
print(f"读取了 {len(transactions)} 条事务")
print(f"前5条事务: {transactions[:5]}")

## 8. 测试不同的分布

In [ ]:
# 比较不同分布
num_items = 100
distributions = ['random', 'zipf', 'normal', 'exponential']

plt.figure(figsize=(14, 10))

for i, dist_method in enumerate(distributions, 1):
    params = {}
    if dist_method == 'zipf':
        params = {'alpha': 1.2}
    elif dist_method == 'normal':
        params = {'mean': 0.5, 'std': 0.15}
    elif dist_method == 'exponential':
        params = {'scale': 0.5}
    
    probs = DistributionEngine.generate_item_frequencies(num_items, dist_method, params)
    
    plt.subplot(2, 2, i)
    plt.bar(range(num_items), probs, alpha=0.7)
    plt.xlabel('物品 ID')
    plt.ylabel('概率')
    plt.title(f'{dist_method.capitalize()} 分布')
    plt.ylim(0, max(probs) * 1.1)

plt.tight_layout()
plt.show()

## 9. 总结

本示例展示了：
1. ✅ 创建和验证配置
2. ✅ 生成合成数据集
3. ✅ 注入特定模式
4. ✅ 查看统计信息
5. ✅ 可视化数据分布
6. ✅ 保存为 SPMF 格式

下一步可以使用生成的数据运行 SPMF 算法进行基准测试！